# Annotate class 1 cells by most similar transcriptome among class 2 and 3 populations

In [1]:
import os,sys
import datetime

### Import scanpy

In [2]:
import scanpy.api as sc
sc.logging.print_versions()
sc.logging.print_memory_usage()
sc.settings.verbosity = 2

scanpy==1.3.4 anndata==0.6.13 numpy==1.15.4 scipy==1.1.0 pandas==0.23.4 scikit-learn==0.20.1 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 
Memory usage: current 0.17 GB, difference +0.17 GB


### Import my utility functions and import statements from github

In [3]:
"""# This cell is run once to download my custom functions and import statements from github

!git clone --depth=1 https://github.com/rapolaszilionis/utility_functions
    
# github doesn't seem to have an option to download a specific version of the repo from the history.
# So I download my utility functions and save the download time by appending it to the directory name.
# These utility functions to be shared together with the notebook.

toappend = datetime.datetime.now().strftime('%y%m%d_%Hh%M')
newname = "utility_functions_%s"%toappend
print(newname)


# rename the py file with utility functions
os.rename("utility_functions",newname)"""

'# This cell is run once to download my custom functions and import statements from github\n\n!git clone --depth=1 https://github.com/rapolaszilionis/utility_functions\n    \n# github doesn\'t seem to have an option to download a specific version of the repo from the history.\n# So I download my utility functions and save the download time by appending it to the directory name.\n# These utility functions to be shared together with the notebook.\n\ntoappend = datetime.datetime.now().strftime(\'%y%m%d_%Hh%M\')\nnewname = "utility_functions_%s"%toappend\nprint(newname)\n\n\n# rename the py file with utility functions\nos.rename("utility_functions",newname)'

In [4]:
# add the utility function folder to PATH
sys.path.append(os.path.abspath("utility_functions_190425_11h32"))

from rz_import_statements import *
import rz_functions as rz
import rz_fig_params # this adjust mpl.rcParams, almost nothing to import
import rz_utility_spring as srz

python version: 3.6.7


# Load data and place into an annData object

In [5]:
!ls data_from_geo

cell_info_8594x25.tsv         injury_barcodes.tsv
control_barcodes.tsv          injury_genes.tsv
control_genes.tsv             injury_loom.loom
control_loom.loom             injury_matrix.mtx
control_matrix.mtx            raw_counts_8594x27998.mtx
gene_names_alphabetically.txt


In [6]:
obspath = 'data_from_geo/cell_info_8594x25.tsv'
countpath = 'data_from_geo/raw_counts_8594x27998.mtx'
genepath = 'data_from_geo/gene_names_alphabetically.txt'

In [7]:
# load cell info
obs = pd.read_csv(obspath,sep='\t',index_col=0)
print(obs.shape)
obs.head()

(8594, 25)


,barcode,condition,total_counts,pass_quality_filters,inj_epithelial,ctr_epithelial,excluded_as_immune_or_mesench,class,population,phase,...,x_control,y_control,x_class3_exploded,y_class3_exploded,x_control_injured,y_control_injured,x_class1_CTR_cell_cyc_removed,y_class1_CTR_cell_cyc_removed,x_class1_INJ_cell_cyc_removed,y_class1_INJ_cell_cyc_removed
0,AAACCTGAGTGCTGCC-1,control,2787,True,False,True,False,class3,ctr_DEEx,G1,...,783.705500,-377.074709,1082.172898,-266.303304,709.78614,-325.505019,NaN,NaN,NaN,NaN
1,AAACCTGAGTGGGTTG-1,control,3325,True,False,True,False,class3,ctr_upper_IEE,G1,...,709.612642,-429.409941,854.230866,-454.025809,529.15516,-471.961493,NaN,NaN,NaN,NaN
2,AAACCTGCAAGTCTAC-1,control,1781,False,False,False,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAACCTGCAATCTGCA-1,control,3468,True,False,False,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAACCTGCACGGTGTC-1,control,1745,False,False,False,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# load counts
adata = sc.read_mtx(countpath)
print(adata.shape)

# add genes (annotation of variables)
adata.var['genes'] = np.loadtxt(genepath,dtype=str)

# make sure var names are genes
adata.var_names = adata.var['genes'].values

# add obs (annotation of observations)
adata.obs = obs

# make sure index is unique AND a string
adata.obs_names_make_unique()
adata.obs_names = adata.obs_names.astype(str)

(8594, 27998)


### Scale (normalize) data

In [9]:
# turn into counts per 10k
print(adata.X[:5,:].sum(axis=1))
print()
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
print(adata.X[:5,:].sum(axis=1))

[[2787.]
 [3325.]
 [1781.]
 [3468.]
 [1745.]]

[[10000.   ]
 [10000.   ]
 [10000.001]
 [10000.   ]
 [ 9999.999]]


## Get centroids (reference transcriptomes)

In [10]:
help(rz.centroids)

Help on function centroids in module rz_functions:

centroids(label, adata, E=None, gene_list=None)
    Calculate average gene expression level per cell label (e.g. cluster).
    input:
        - label: name of column that stores the label of interest in adata.obs
        - adata: AnnData object OR a cell x feature pandas dataframe with label as one of the columns
        - E and gene_list: optional and only used when adata is not an AnnData object. In that case
        the cells x genes sparse expression matrix E and the gene_list must be specified
        
    returns:
        pandas dataframe, centroids x genes



In [11]:
label = 'population'
cell_mask = adata.obs['ctr_epithelial'].astype(bool).values
centroids = rz.centroids(label,adata[cell_mask])

In [12]:
centroids.head()

,0610007P14Rik,0610009B22Rik,0610009L18Rik,0610009O20Rik,0610010F05Rik,0610010K14Rik,0610011F06Rik,0610012D04Rik,0610012G03Rik,0610025J13Rik,...,mt-Co2,mt-Co3,mt-Cytb,mt-Nd1,mt-Nd2,mt-Nd3,mt-Nd4,mt-Nd4l,mt-Nd5,mt-Nd6
ctr_DEEx,1.391852,0.879937,0.324726,0.209586,0.086968,0.329960,0.954454,0.0,1.200782,0.0,...,14.079573,83.904465,56.074482,23.852005,3.198850,1.030915,18.962263,0.987729,1.261443,0.275973
ctr_upper_IEE,1.300044,0.695099,0.314438,0.287892,0.184926,0.397997,0.773451,0.0,0.998683,0.0,...,15.316485,84.531807,54.308296,24.424025,3.601944,1.055761,18.239050,0.895775,0.950987,0.232046
ctr_M_G1,1.303535,0.552314,0.065731,0.330279,0.126810,0.566542,0.606099,0.0,1.455639,0.0,...,12.348806,70.209267,47.538788,23.879992,2.751078,0.998057,15.154018,0.907756,0.761977,0.202841
ctr_OEE_IEE,1.737094,0.544913,0.267885,0.461709,0.105900,0.447238,0.927448,0.0,1.360467,0.0,...,13.681261,85.969254,55.733368,22.308928,4.568748,0.957020,19.031935,0.749691,1.071926,0.076110
ctr_G2_M,2.209283,0.774067,0.063674,0.473809,0.182266,0.590541,0.573025,0.0,1.316742,0.0,...,12.441298,70.319290,45.029736,23.863428,3.214058,0.953022,15.301864,0.760218,1.103230,0.170797


## Run classifier with 100 most enriched genes per class2,3 population

In [13]:
# select cells to classify
toclassify = adata.obs['class'].isin(['class1'])&\
            (adata.obs['ctr_epithelial'].astype(bool)|adata.obs['inj_epithelial'].astype(bool))
toclassify = toclassify.values
print(toclassify.sum(),len(toclassify))
E = adata[toclassify].X
gene_list = adata.var_names

# reference transcriptomes to classify by
category_labels = [i for i in centroids.index if i not in adata[toclassify].obs['population'].unique()]
categories = centroids.T[category_labels]
print(category_labels)

3344 8594
['ctr_DEEx', 'ctr_upper_IEE', 'ctr_OEE_IEE', 'ctr_OEE_2', 'ctr_OSR', 'ctr_SI', 'ctr_VEE', 'ctr_pre_AMB', 'ctr_OEE_1', 'ctr_AMB_dist', 'ctr_AMB_prox', 'ctr_ISR_SI']


In [14]:
# load table with top 100 most enriched genes per population
top100 = pd.read_excel('supplementary_tables_Sharir_et_al_2019/Table_S1.xlsx',header=1)

top100.columns = [i.replace('/','_').replace(' ','_').replace('-','_') for i in top100.columns]
top100.columns = ['ctr_'+i for i in top100.columns]
top100.head()

,ctr_DEEx,ctr_DEEx_FC,ctr_upper_IEE,ctr_upper_IEE_FC,ctr_M_G1,ctr_M_G1_FC,ctr_OEE_IEE,ctr_OEE_IEE_FC,ctr_G2_M,ctr_G2_M_FC,...,ctr_pre_AMB,ctr_pre_AMB_FC,ctr_OEE_1,ctr_OEE_1_FC,ctr_AMB_dist,ctr_AMB_dist_FC,ctr_AMB_prox,ctr_AMB_prox_FC,ctr_ISR_SI,ctr_ISR_SI_FC
0,Dcn,3.614268,Rnaset2a,2.148060,Cenpa,3.363334,Cldn10,5.468689,Ube2c,7.981552,...,Nudt4,2.511703,Grp,7.475961,Ambn,79.594589,Amelx,40.556911,Krt17,5.546733
1,Aldh1a3,3.171303,Phlda1,1.994479,Hist1h2bc,3.181940,Pthlh,4.153130,Top2a,6.105304,...,Iqgap2,2.222442,Nppc,6.380324,Enam,44.962585,Calb1,37.223640,Tacstd2,4.155315
2,Sfrp5,2.575278,Ifitm3,1.992567,Ccnb2,2.390234,Grp,3.896335,Kpna2,5.745326,...,Satb1,2.204441,Fosb,6.052104,Amelx,41.222912,Enam,35.814861,Tagln,3.631651
3,Ccnd2,2.470626,Sox4,1.895130,Hmgb2,2.380920,Crabp1,3.029581,Nusap1,5.553847,...,Col27a1,2.184898,Dcn,4.905131,Clu,39.412006,Clu,23.112156,Igfbp3,3.444902
4,En1,2.386213,Fam19a4,1.858609,Lockd,2.336322,Shisa2,2.936753,Ccnb1,5.493615,...,Vwde,2.173148,Tgfb2,4.904398,Mmp20,14.765478,Plod2,22.421888,Slc39a10,3.411926


In [15]:
# enriched genes in clusters outside of cycling cells
enriched_genes = top100[category_labels].values.flatten()
labels_to_avoid = [i for i in centroids.index if i not in category_labels]
genes_to_avoid = top100[labels_to_avoid].values.flatten()
print(len(enriched_genes))
enriched_genes = list(set(enriched_genes))
print(len(enriched_genes))
# exclude occasional cell cycle gene
enriched_genes = [i for i in enriched_genes if i not in genes_to_avoid]
print(len(enriched_genes))

1200
800
790


In [16]:
# Select genes to use, prepare boolean mask.
genes_to_use = enriched_genes
gene_mask = np.in1d(gene_list,genes_to_use)

pseudo = 1

In [17]:
start = time.time()
bays = []
i = 0
interval = 1000
for j in range(interval,E.shape[0]+interval,interval):
    j = min(j,E.shape[0])
    stepsize = j-i
    tmp_dense = pd.DataFrame(E.T[gene_mask][:,i:j].todense())
    tmp_dense.index = np.array(gene_list)[gene_mask]
    bay = rz.bayesian_classifier(tmp_dense,categories.loc[tmp_dense.index]+pseudo)
    bays.append(bay)
    i = j
    print('%.2f min.'%((time.time()-start)/60.))
    print('cells from %d to %d done'%(i-stepsize,min(i,E.shape[0])))
    
bay = pd.concat(bays,axis=1)
bay.columns = np.arange(bay.shape[1])

0.00 min.
cells from 0 to 1000 done
0.01 min.
cells from 1000 to 2000 done
0.01 min.
cells from 2000 to 3000 done
0.01 min.
cells from 3000 to 3344 done


In [18]:
# log likelihoods. Can be use directly to find maximum likelihood.
bay

,0,1,2,3,4,5,6,7,8,9,...,3334,3335,3336,3337,3338,3339,3340,3341,3342,3343
ctr_DEEx,-4031.168457,-3819.042480,-3348.066895,-4054.513184,-3619.425293,-5047.666016,-3317.287109,-4297.836426,-3743.825195,-3645.994629,...,-3718.858398,-4774.810547,-3405.502441,-3209.570312,-3864.764893,-3724.276855,-4436.709473,-4342.840820,-4401.244629,-4121.519531
ctr_upper_IEE,-4018.796631,-3802.652832,-3302.541504,-4000.065918,-3571.762207,-5118.182617,-3299.607666,-4274.031250,-3733.900879,-3626.227539,...,-3714.743164,-4835.724609,-3373.500000,-3220.977295,-3836.332520,-3723.338379,-4413.879883,-4315.059082,-4414.622070,-4197.270508
ctr_OEE_IEE,-4082.834961,-3848.474365,-3354.785156,-3944.142578,-3615.201172,-5163.550781,-3331.181641,-4298.198242,-3771.521729,-3695.071777,...,-3721.506104,-4784.722656,-3469.545166,-3257.449463,-3906.003662,-3803.436523,-4337.062500,-4443.498047,-4507.806152,-4243.020020
ctr_OEE_2,-4123.669922,-3926.993408,-3445.233398,-4103.208984,-3727.003418,-5001.466797,-3432.708984,-4385.477539,-3836.349854,-3766.377686,...,-3784.227539,-4701.400391,-3548.579102,-3312.280762,-3954.685791,-3856.008545,-4407.008789,-4508.538086,-4547.631836,-4244.314453
ctr_OSR,-4073.936035,-3880.292969,-3368.363525,-4003.779785,-3695.619385,-5140.972168,-3396.118164,-4286.783203,-3801.904053,-3734.900635,...,-3771.967529,-4659.078613,-3516.998535,-3315.391846,-3826.553467,-3841.211670,-4255.573242,-4489.912598,-4537.487305,-4261.460938
ctr_SI,-4040.829102,-3855.736328,-3334.332520,-4002.202148,-3623.750000,-5152.177734,-3362.491699,-4231.451172,-3749.849609,-3664.914307,...,-3748.192139,-4702.145996,-3454.336426,-3279.175781,-3783.816895,-3782.928955,-4315.301758,-4397.071289,-4464.516113,-4236.380859
ctr_VEE,-4140.198242,-3991.459229,-3465.568848,-4114.451172,-3776.950684,-5093.940918,-3489.823242,-4465.648438,-3860.351074,-3780.692383,...,-3832.022949,-4754.228516,-3553.413574,-3355.746582,-3998.401367,-3882.810303,-4483.126953,-4555.071289,-4568.892578,-4286.059570
ctr_pre_AMB,-3971.533691,-3822.525391,-3348.704102,-4074.522949,-3575.081055,-5142.391602,-3328.959717,-4333.919922,-3689.512207,-3595.837646,...,-3756.101562,-4803.732422,-3413.587402,-3270.857422,-3883.302246,-3745.017822,-4435.564453,-4301.487305,-4366.380371,-4199.173828
ctr_OEE_1,-4168.995117,-3952.783447,-3490.737549,-4115.011230,-3746.059326,-5191.430664,-3425.160156,-4422.537109,-3849.865967,-3806.821045,...,-3823.564941,-4748.683594,-3615.694824,-3367.099365,-4038.809570,-3922.755127,-4444.573242,-4587.330078,-4635.699219,-4315.931152
ctr_AMB_dist,-4517.004883,-4381.872070,-3891.899414,-4596.195801,-4184.358887,-5787.214844,-3891.475586,-4949.999023,-4254.697266,-4150.666504,...,-4347.254883,-5462.891602,-3977.195801,-3789.163086,-4452.683594,-4317.268066,-5040.771484,-4979.303223,-5000.744629,-4838.551270


In [19]:
# maximum likelihood for every cell
recreated = bay.idxmax().apply(lambda x: x+'_like').values
recreated

array(['ctr_pre_AMB_like', 'ctr_upper_IEE_like', 'ctr_upper_IEE_like',
       ..., 'ctr_pre_AMB_like', 'ctr_pre_AMB_like', 'ctr_DEEx_like'],
      dtype=object)

## Compare to results provided on GEO

In [20]:
on_geo = adata[toclassify].obs['class2_3_like'].values

In [21]:
mismatches = recreated!=on_geo
print("%d out of %d cells have mismatching labels"%(mismatches.sum(),len(mismatches)))

0 out of 3344 cells have mismatching labels
